In [69]:
import pandas
import sklearn

In [70]:
train = pandas.read_csv("train.csv")
test = pandas.read_csv("test.csv")

In [71]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [72]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [73]:
test = test.set_index("PassengerId")
train = train.set_index("PassengerId")

In [74]:
y = train["Survived"]

In [75]:
y.head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

In [76]:
columns = ["Pclass", "Sex", "Age", "Fare"]
X = train[columns]

# TODO use an actual encoder
X.loc[X["Sex"] == "male", :] = 1
X.loc[X["Sex"] == "female", :] = 0

/home/eric/.cache/pypoetry/virtualenvs/kaggle-rP8LzGqY-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## First attempt; baby logistic regression model

In [15]:
import sklearn.linear_model

logistic_regression = sklearn.linear_model.LogisticRegression(random_state=0)

In [24]:
import sklearn.model_selection

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

In [18]:
X_train.shape

(668, 5)

In [19]:
y_train.shape

(668,)

In [35]:
import sklearn.metrics
import sklearn.model_selection


X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)
logistic_regression = sklearn.linear_model.LogisticRegression(random_state=0)
logistic_regression.fit(X_train, y_train)
sklearn.metrics.accuracy_score(logistic_regression.predict(X_test), y_test)

0.7937219730941704

### Parametrize it

In [37]:
import pandas._typing
import sklearn.base
import sklearn.ensemble
import sklearn.model_selection


def train_test_lin_reg(
        X: pandas._typing.ArrayLike,
        y: pandas._typing.ArrayLike,  # TODO score column as a param and pass full DF?
        # TODO model as a param?
        # TODO scorer as a param?
    ) -> (sklearn.base.BaseEstimator, float):
    """
    
    Args:
        X: Full training set
        y: Full test set
    
    Returns:
        Tuple of the estimator and the accuracy.
    """
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)
    logistic_regression = sklearn.linear_model.LogisticRegression(random_state=0)
    logistic_regression.fit(X_train, y_train)
    accuracy = sklearn.metrics.accuracy_score(logistic_regression.predict(X_test), y_test)
    return logistic_regression, accuracy

In [38]:
estimator, accuracy = train_test_lin_reg(X, y)
print(accuracy)

0.8026905829596412


### Parametrize a little more...

In [45]:
import pandas._typing
import sklearn.base
import sklearn.ensemble
import sklearn.model_selection


def train_test_model(
        X: pandas._typing.ArrayLike,
        y: pandas._typing.ArrayLike,  # TODO score column as a param and pass full DF?
        model: sklearn.base.BaseEstimator,
        # TODO scorer as a param?
    ) -> (sklearn.base.BaseEstimator, float):
    """
    
    Args:
        X: Full training set
        y: Full test set
    
    Returns:
        Tuple of the trained estimator and the accuracy.
    """
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=0)
    model.fit(X_train, y_train)
    accuracy = sklearn.metrics.accuracy_score(model.predict(X_test), y_test)
    return model, accuracy

In [46]:
estimator, accuracy = train_test_model(X, y, sklearn.linear_model.LogisticRegression())
print(accuracy)

0.7802690582959642


In [77]:
estimator, accuracy = train_test_model(X, y, sklearn.ensemble.RandomForestClassifier(random_state=0))
print(accuracy)

0.7802690582959642


## Now let's try running it on the test set

In [78]:
X_test = test[columns]
X_test.loc[X_test["Sex"] == "male", :] = 1
X_test.loc[X_test["Sex"] == "female", :] = 0
preds = estimator.predict(X_test)

/home/eric/.cache/pypoetry/virtualenvs/kaggle-rP8LzGqY-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [79]:
preds[:10]

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0])

In [80]:
X_test.head()

,Pclass,Sex,Age,Fare
PassengerId,,,,
892,1,1,1.0,1.0
893,0,0,0.0,0.0
894,1,1,1.0,1.0
895,1,1,1.0,1.0
896,0,0,0.0,0.0


In [81]:
outputs = X_test.copy()

In [82]:
outputs.shape, preds.shape

((418, 4), (418,))

In [87]:
outputs["Survived"] = preds
outputs = outputs["Survived"]
outputs.head()

PassengerId
892    0
893    1
894    0
895    0
896    1
Name: Survived, dtype: int64

In [88]:
outputs.to_csv("preds.csv")

In [89]:
!head "preds.csv"

PassengerId,Survived
892,0
893,1
894,0
895,0
896,1
897,0
898,1
899,0
900,1
